THINGS TO DO (IN ORDER)
- import everything
- get the data
- analyzze data
  - oversample if needed
  - clean up anything wierd (there shoudlnt be much)
- split the data (train adn test)
- training data
  - find best parameters for this / different models
- test teh data
- get all reselut s
- map results

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
%config InlineBackend.figure_format="retina"
import numpy as np
import random
import torch
from torch import nn, optim
import math
from IPython import display
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset
import pdb
from sklearn.neural_network import MLPClassifier
from sklearn import model_selection
from sklearn import metrics
from collections import Counter
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from PIL import Image
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error



In [ ]:
traindf = pd.read_csv("_classes.csv")
traindf.describe()
traindf[""]

,blackheads,dark spot,nodules,papules,pustules,whiteheads
count,966.000000,966.000000,966.000000,966.000000,966.000000,966.000000
mean,0.214286,0.527950,0.123188,0.562112,0.312629,0.274327
std,0.410538,0.499477,0.328824,0.496384,0.463805,0.446406
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000
75%,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
